# Notebook for various charts posted on @chickenandstats

## 0. Housekeeping

### Import dependencies

In [ ]:
import pandas as pd
import numpy as np

from my_module.sql import read_sql
from my_module.pd import pd_options
from hockey_rink import NHLRink

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
import matplotlib.ticker as mtick
import matplotlib.animation as animation
from matplotlib.lines import Line2D
import matplotlib.patches as patches
import matplotlib.patheffects as mpe

from celluloid import Camera

from tqdm.auto import tqdm

from chickenstats.chicken_nhl import scrape_schedule, scrape_boxscore, scrape_api_events, NHL_COLORS, scrape_html_rosters

### Setting pandas options

In [ ]:
# Updating pandas settings for maximum rows and columns displayed

pd.set_option('max_rows', 150)
pd.set_option('max_columns', None)


## 1. Data reading & munging

### Reading from CSV

In [ ]:
# Reading evolving-hockey pbp, individual on-ice, and individual game stats dataframes
# Individual on-ice and game stats are munged from EH pbp

folder = './data/'

eh_ind = pd.read_csv(folder + 'evolving_hockey_ind.csv', index_col = 0, low_memory = False)
eh_oi = pd.read_csv(folder + 'evolving_hockey_oi.csv', index_col = 0, low_memory = False)
eh_pbp = pd.read_csv(folder + 'evolving_hockey_pbp.csv', index_col = 0, low_memory = False)
#rosters = pd.read_csv(folder + 'rosters.csv', index_col = 0, low_memory = False

### Function to aggregate individual statistics to overall game and strength level

In [ ]:
def agg_ind_stats(data):
    
    '''
    Function to aggregate individual box score statistics
        
    Returns two dataframes, one at the strength and game level, the other
    at the game level
    
    Arguments
        data: dataframe
    '''
    
    df = data.copy()
    
    # Creating datetime column
    
    df['game_date_dt'] = pd.to_datetime(df.game_date, format = '%Y-%m-%d')
    
    # Grouping for strength and game level
    
    group_list = ['season', 'session', 'game_id', 'game_date', 'game_date_dt',
                  'player', 'team', 'strength_state']
    
    # Statistics to aggregate

    stats = ['G', 'A1', 'A2', 'iSF', 'iFF','iCF', 'ixG', 'GaX',
             'missed_shots', 'shots_blocked_off', 'GIVE', 'TAKE',
             'iHF', 'iHT', 'FOW', 'FOL', 'A1_xG', 'A2_xG', 'iPENT2',
             'iPENT4', 'iPENT5', 'iPENT10', 'iPEND2', 'iPEND4',
             'iPEND5', 'iPEND10', 'iPENT0', 'iPEND0']
    
    # Dictionary to be used for aggregating with groupby

    stats = {x: 'sum' for x in stats}
    
    # List by which to sort the finished dataframe

    sort_list = ['game_date_dt', 'game_id', 'team', 'player', 'strength_state']
    
    # Creating stength and game level dataframe

    ind_strength = df.groupby(group_list, as_index = False).agg(stats)\
                        .sort_values(by = sort_list)\
                        .reset_index(drop = True)
    
    # Grouping for game level dataframe
    
    group_list = ['season', 'session', 'game_id', 'game_date', 'game_date_dt',
                  'player', 'team']
    
    # Statistics to aggregate

    stats = ['G', 'A1', 'A2', 'iSF', 'iFF','iCF', 'ixG', 'GaX',
             'missed_shots', 'shots_blocked_off', 'GIVE', 'TAKE',
             'iHF', 'iHT', 'FOW', 'FOL', 'A1_xG', 'A2_xG', 'iPENT2',
             'iPENT4', 'iPENT5', 'iPENT10', 'iPEND2', 'iPEND4',
             'iPEND5', 'iPEND10', 'iPENT0', 'iPEND0']
    
    # Dictionary to be used with aggregating with groupby

    stats = {x: 'sum' for x in stats}
    
    # List by which to sort finished dataframe

    sort_list = ['game_date_dt', 'game_id', 'team', 'player']
    
    # Creating game level dataframe

    ind_game = df.groupby(group_list, as_index = False).agg(stats)\
                    .sort_values(by = sort_list)\
                    .reset_index(drop = True)
    
    return ind_strength, ind_game
    

### Function to aggregate and combine individual and on-ice statistics

In [ ]:
def game_s_func(ind, oi):
    
    '''
    Function to aggregate individual and on-ice stats to game and strength level,
    then combine them
    '''
    
    ind = ind.copy()
    
    oi = oi.copy()
    
    # Individual stats to aggregate
    
    ind_stats = ['G', 'iSF', 'iCF', 'iFF', 'ixG', 'GaX']
    
    # Dictionary used in groupby aggregaton

    agg_dict = {x: 'sum' for x in ind_stats}
    
    # Mapping to group by

    group_list = ['season', 'session', 'game_id', 'game_date', 'team',
                  'opp_team', 'player', 'strength_state']
    
    # Creating individual stats dataframe

    game_s_ind = ind.groupby(group_list, as_index = False).agg(agg_dict)
    
    # On ice stats to aggregate
    
    oi_stats = ['GF', 'SF', 'CF', 'FF', 'xGF', 'GA', 'SA', 'CA', 'FA', 'xGA', 'TOI', 'OZF', 'DZF', 'NZF', 'FAC']
    
    # Dictionary used in groupby aggregation

    agg_dict = {x: 'sum' for x in oi_stats}
    
    # Mapping to group by

    group_list = ['season', 'session', 'game_id', 'game_date', 'team',
                  'opp_team', 'player', 'strength_state']
    
    # Creating on ice dataframe

    game_s_oi = eh_oi.groupby(group_list, as_index = False).agg(agg_dict)
    
    # Merging dataframes
    
    game_s = game_s_oi.merge(game_s_ind, on = group_list, how = 'left').fillna(0)
    
    # Creating percentage columns
    
    game_s['xgf_perc'] = game_s.xGF / (game_s.xGF + game_s.xGA)

    game_s['ozf_perc'] = game_s.OZF / game_s.FAC
    
    return game_s

    
    

### Munging data

In [ ]:
ind_strength, ind_game = agg_ind_stats(eh_ind)

game_s = game_s_func(eh_ind, eh_oi)

## 2. Individual career rink map plots

### Function to normalize coords for plots

In [ ]:
def norm_coords(data, norm_player):
    '''
    Function to normalize coordinates to one zone
    
    Attributes
        data: a one-game dataframe
        
        norm_player: the player's name to normalize
        
    '''
    
    # Definining the player filter conditions
    
    good_player = data.event_player_1 == norm_player
    bad_player = data.event_player_1 != norm_player
    
    # Defining the location filter conditions
    
    neg_x = (data['coords_x'] < 0)
    pos_x = (data['coords_x'] > 0)
    
    # Combining the conditions into a list for np.select
    
    conditions = [good_player & neg_x, bad_player & pos_x]
    
    # Defining the values for each condition in np.select
    values_x = [data['coords_x'].mul(-1), data['coords_x'].mul(-1)]
    values_y = [data['coords_y'].mul(-1), data['coords_y'].mul(-1)]

    # Creating the normalized coordinate columns
    
    data['norm_coords_x'] = np.select(conditions, values_x, default = data['coords_x'])
    data['norm_coords_y'] = np.select(conditions, values_y, default = data['coords_y'])
    
    return data

### Function to create the individual player pbp dataframe

In [ ]:
def get_player_pbp(data, player, strengths):
    
    '''
    Function to get individual player pbp from EH pbp
    
    Arguments
        player: player name - str
        strengths: strength states - list
        session: 'R', 'P', or both - list
    
    '''
    
    # Create copy of data set
    
    eh_df = data.copy()
    
    # Create conditions for filtering data set
    
    event_list = ['GOAL', 'SHOT', 'MISS']
    
    conds = [eh_df.event_type.isin(event_list),
             eh_df.event_player_1 == player, 
             eh_df.strength_state.isin(strengths)]
    
    # Filtering data set
    
    player_pbp = eh_df[np.logical_and.reduce(conds)].copy().reset_index(drop = True)
    
    # Normalizing coordinates based on the player
    
    player_pbp = norm_coords(player_pbp, player)
    
    return player_pbp
    

### Function to create individual player game stats dataframe

In [ ]:
def get_player_ind(data, player, strengths):
    
    '''
    Function to get the proper individual player stats
    '''
    
    df = data.copy()
    
    # Stats to aggregate
    
    stats = ['G', 'A1', 'A2', 'iSF', 'iFF', 'iCF', 'ixG',
             'GaX', 'missed_shots', 'shots_blocked_off',
             'GIVE', 'TAKE', 'iHF', 'iHT', 'FOW', 'FOL',
             'A1_xG', 'A2_xG', 'iPENT2', 'iPENT4', 'iPENT5',
             'iPENT10', 'iPEND2', 'iPEND4', 'iPEND5', 'iPEND10',
             'iPENT0', 'iPEND0']
    
    # Creating dictionary to be used in groupby aggregation
    
    stats = {x: 'sum' for x in stats}
    
    # Conditions for filtering
    
    conds = [df.player == player, 
             df.strength_state.isin(strengths)]
    
    conds = np.logical_and.reduce(conds)
    
    # Grouping for groupby
    
    group_list = ['season', 'session', 'player']
    
    # Creating player dataframe
    
    player_ind = df[conds].groupby(group_list, as_index = False).agg(stats)
    
    return player_ind
    

### Function to get TOI from on-ice stats

In [ ]:
def get_player_oi(data, player, strengths):
    
    '''
    Function to get on-ice data for a given player in given situations
    '''
    
    df = data.copy()
    
    # Stats to aggregate
    
    stats = ['BSF', 'GF', 'HF', 'MSF', 'PENT2', 'PENT4', 'PENT5',
             'PENT10', 'SF', 'CF', 'FF', 'xGF', 'OZFW', 'DZFW',
             'NZFW', 'BSA', 'GA', 'HT', 'MSA', 'PEND2', 'PEND4',
             'PEND5', 'PEND10', 'SA', 'CA', 'FA', 'xGA', 'DZFL',
             'OZFL', 'NZFL', 'TOI', 'OZF', 'DZF', 'NZF', 'FAC',
             'PENT0', 'PEND0']
    
    # Dictionary to be used with groupby aggregation
    
    stats = {x: 'sum' for x in stats}
    
    # Conditions for filtering
    
    conds = [df.player == player, 
             df.strength_state.isin(strengths)]
    
    conds = np.logical_and.reduce(conds)
    
    # Grouping for groupby
    
    group_list = ['player', 'season', 'session']
    
    # Creating dataframe
    
    player_oi = df[conds].groupby(group_list, as_index = False).agg(stats)
    
    return player_oi
    

### Function get unique years for plot

In [ ]:
def get_years(data, session):
    
    '''
    Function to get a list of years from player pbp dataframe
    '''
    
    df = data.copy()
    
    # Filter conditions
    
    conds = df.session == session
    
    years = sorted(df.season.unique().tolist())
    
    return years

### Individual career rink maps

In [ ]:
player = 'ROMAN.JOSI'

player = 'RYAN.JOHANSEN'

player = 'FILIP.FORSBERG'

#player = 'MATT.DUCHENE'

#strengths = ['5v4', '5v3', '4v3']

strengths = ['5v5']

#strengths = ['5v4', '5v3', '4v3']

session = 'R'

player_pbp = get_player_pbp(eh_pbp, player, strengths)

player_ind = get_player_ind(eh_ind, player, strengths)

player_oi = get_player_oi(eh_oi, player, strengths)

years = get_years(player_pbp, session)

In [ ]:

# Set color scheme

colors = NHL_COLORS['NSH']

# Creating rink instance

rink = NHLRink(rotation = 90)

# Generating subplot arguments

if len(years) > 10:
    
    nrows = 4
    ncols = 3
    figsize = (6, 8)
    
if len(years) == 9 or len(years) == 10:
    
    nrows = 5
    ncols = 2
    figsize = (4, 10)
    
if len(years) == 7 or len(years) == 8:
    
    nrows = 4
    ncols = 2
    figsize = (4, 8)
    
if len(years) == 5 or len(years) == 6:
    
    nrows = 3
    ncols = 2
    figsize = (4, 6)
    
# Generating figure and subplots

fig, axes = plt.subplots(nrows = nrows, ncols = ncols, figsize = figsize, dpi = 500)

# Reshaping axes to make it easier to iterate through them

axes = axes.reshape(-1)

for idx, ax in enumerate(axes):
    
    # Setting year. If doesn't exist, making the ax invisible
    
    try:
    
        year = np.flip(years)[idx]
        
    except IndexError:
        
        ax.set_visible(False)
        
        continue
        
    # Drawing the rink

    rink.draw(display_range = 'ozone', ax = ax)
    
    # Iterating through each event and plotting them

    for event in ['MISS', 'SHOT', 'GOAL']:

        plot_df = player_pbp[np.logical_and(player_pbp.season == year, player_pbp.event_type == event)]
        
        # Seetting the edge color

        if event == 'GOAL':

            edge_color = colors['SHOT']

        else:

            edge_color = colors[event]
            
        # Setting alpha

        alpha = 0.8
        
        #Plotting the event data

        rink.scatter(x = plot_df.norm_coords_x, y = plot_df.norm_coords_y, ax = ax, color = colors[event],
                     edgecolors = edge_color, alpha = alpha, s = plot_df.pred_goal * 100)
        
    # Getting situational TOI and statistics
        
    toi = player_oi[np.logical_and(player_oi.season == year, player_oi.session == session)].TOI.iloc[0]
    
    stats = player_ind[np.logical_and(player_ind.season == year, player_ind.session == session)]
    
    g = stats.G.iloc[0]
    
    xg = stats.ixG.iloc[0]
    
    # Setting ax title and subtitle
    
    title_str = f'{str(year)[:4]}-{str(year)[-4:]}'
    
    ax.set_title(title_str, weight = 'bold', fontsize = 6, ha = 'center', y = 1.03)
                 
    sub_t_str = f'{round(g, 2)} G, {round(xg, 2)} xG, & {round(toi, 2)} TOI'
    
    ax.text(s = sub_t_str, x = .5, y = 1.025, fontsize = 5, ha = 'center', transform=ax.transAxes)
    
    # Setting the legend elements
    
    legend_elements = list()
    
    for result, color in colors.items():

        if result == 'GOAL':

            edge_color = colors['SHOT']

        else:

            edge_color = color

        element = Line2D([0], [0], markeredgecolor = edge_color, marker = 'o', markerfacecolor = color,
                    label = result, markersize = 4, color = 'w', alpha = 0.8)

        legend_elements.append(element)
        
    # Plotting the legend

    legend = ax.legend(handles = legend_elements, loc="lower center", ncol=3, fontsize = 4,
                       facecolor = 'white', framealpha = 1,
                       edgecolor = 'gray')
    
# Setting title and filepath names based on situation

if strengths == ['5v5']:
    
    strength_t = '5v5'
    strength_f = strength_t
    
if strengths == ['5v4', '5v3', '4v3']:
    
    strength_t = 'Power play'
    strength_f = 'pp'
    
if strengths == ['5v5', '4v4', '3v3']:
    
    strength_t = 'Even strength'
    strength_f = 'ff'
    
# Title strings
    
st_str = f"{player.replace('.', ' ')} CAREER SHOT MAPS"

sub_str = f"{strength_t} situations | Data @EvolvingHockey | Viz @chickenandstats"

# Moving title around based on figsize

if len(years) > 10:
    
    fig.suptitle(st_str, y = .93, x = .5, ha = 'center', fontsize = 8, weight = 'bold')

    fig.text(s = sub_str, x = .5, y = .907, ha = 'center', fontsize = 6)
    
else:
    
    fig.suptitle(st_str, y = .925, x = .5, ha = 'center', fontsize = 8, weight = 'bold')

    fig.text(s = sub_str, x = .5, y = .905, ha = 'center', fontsize = 6)
    
# Saving figure

file_path = f"./charts/ind_maps/{player.replace('.', '_').lower()}_map_{strength_f}.png"

#fig.savefig(file_path, bbox_inches = 'tight', dpi = 500, facecolor = 'white') #commented out


## 3. Team powerplay rink maps

In [ ]:
# Setting conditions for filtering

season = 20212022

session = 'R'

strengths = ['5v4', '4v3', '5v3']

pp_players = ['ROMAN.JOSI', 'MIKAEL.GRANLUND', 'MATT.DUCHENE', 
              'RYAN.JOHANSEN', 'FILIP.FORSBERG', 'EELI.TOLVANEN', 
              'MATTIAS.EKHOLM', 'NICK.COUSINS', 'PHILIP.TOMASINO']

team = 'NSH'

conds = [eh_pbp.event_player_1.isin(pp_players),
         eh_pbp.strength_state.isin(strengths),
         eh_pbp.season == season,
         eh_pbp.session == session]

conds = np.logical_and.reduce(conds)

# Filtering

plot_df = eh_pbp[conds]

# Set color scheme

colors = NHL_COLORS[team]

# Creating rink instance

rink = NHLRink(rotation = 90)
    
# Generating figure and subplots

fig, axes = plt.subplots(nrows = 3, ncols = 3, figsize = (6, 5.5), dpi = 500)

# Reshaping axes to make it easier to iterate through them

axes = axes.reshape(-1)

# Iterating through axes

for idx, ax in enumerate(axes):
    
    # Setting player
    
    player = pp_players[idx]
    
    # Getting player pbp data
    
    player_pbp = get_player_pbp(plot_df, player, strengths)
    
    # Filtering game stats
    
    conds = [game_s.session == 'R',
             game_s.season == season,
             game_s.player == player,
             game_s.strength_state.isin(strengths)]
    
    conds = np.logical_and.reduce(conds)

    player_ind = game_s[conds]
        
    # Drawing the rink

    rink.draw(display_range = 'ozone', ax = ax)
    
    # Iterating through each event and plotting them

    for event in ['MISS', 'SHOT', 'GOAL']:

        plot_ = player_pbp[player_pbp.event_type == event]
        
        # Seetting the edge color

        if event == 'GOAL':

            edge_color = colors['SHOT']

        else:

            edge_color = colors[event]
            
        # Setting alpha

        alpha = 0.8
        
        #Plotting the event data

        rink.scatter(x = plot_.norm_coords_x, y = plot_.norm_coords_y, ax = ax, color = colors[event],
                     edgecolors = edge_color, alpha = alpha, s = plot_.pred_goal * 100)
        
    # Getting situational TOI and statistics
        
    toi = player_ind.TOI.sum()
    
    #stats = player_ind[np.logical_and(player_ind.season == year, player_ind.session == session)]
    
    g = player_ind.G.sum()
    
    xg = player_ind.ixG.sum()
    
    # Setting ax title and subtitle
    
    title_str = f"{player.replace('.', ' ')}"
    
    ax.set_title(title_str, weight = 'bold', fontsize = 6, ha = 'center', y = 1.03)
                 
    sub_t_str = f'{round(g)} G, {round(xg, 2)} xG, & {round(toi, 2)} TOI'
    
    ax.text(s = sub_t_str, x = .5, y = 1.025, fontsize = 5, ha = 'center', transform=ax.transAxes)
    
    # Setting the legend elements
    
    legend_elements = list()
    
    for result, color in colors.items():

        if result == 'GOAL':

            edge_color = colors['SHOT']

        else:

            edge_color = color

        element = Line2D([0], [0], markeredgecolor = edge_color, marker = 'o', markerfacecolor = color,
                    label = result, markersize = 4, color = 'w', alpha = 0.8)

        legend_elements.append(element)
        
    # Plotting the legend

    legend = ax.legend(handles = legend_elements, loc="lower center", ncol=3, fontsize = 4,
                       facecolor = 'white', framealpha = 1,
                       edgecolor = 'gray')
    
# Title strings

st_str = 'Power play scoring driven by top unit'.upper()

stsb_str = f'2021-22 {team} power play rink maps | Data @EvolvingHockey | Viz @chickenandstats'
    
fig.suptitle(st_str, ha = 'center', va = 'center', x = 0.5, y = .95, fontsize = 7, weight = 'bold')

fig.text(s = stsb_str, ha = 'center', va = 'center', x = .5, y = .93, fontsize = 6)
    
# Saving figure

file_path = f"./charts/pp_maps/{team}_map_pp.png"

#fig.savefig(file_path, bbox_inches = 'tight', dpi = 500, facecolor = 'white') #commented out


## 4. Usage and offense scatter plots

### Function to filter data

In [ ]:
def prep_usage(data, season, session, strengths, team):
    
    '''
    Function to filter the game strength data to the proper year, session, and team
    '''
    
    df = data.copy()
    
    # Conditions for filtering
    
    conds = [df.season == season,
             df.session == session,
             df.strength_state.isin(strengths),
             df.team == team]
    
    conds = np.logical_and.reduce(conds)
    
    # Filtering dataframe
    
    df = df[conds].reset_index(drop = True)
    
    return df

### Prepping data

In [ ]:
game_s = game_s_func(eh_ind, eh_oi)

### Plotting data

In [ ]:
season = 20212022
session = 'R'
strengths = ['5v5']
team = 'NSH'

plot_df = prep_usage(game_s, season, session, strengths, team)

player_list = ['MATT.DUCHENE', 'MIKAEL.GRANLUND', 'YAKOV.TRENIN',
               'TANNER.JEANNOT', 'COLTON.SISSONS', 'LUKE.KUNIN', 
               'RYAN.JOHANSEN', 'FILIP.FORSBERG', 'EELI.TOLVANEN',
               'PHILIP.TOMASINO', 'NICK.COUSINS', 'MICHAEL.MCCARRON']

# Setting color scheme

colors = NHL_COLORS['NSH']

# Generating fig and subplots

fig, axes = plt.subplots(nrows = 4, ncols = 3, figsize = (8, 8), dpi = 500)

# Reshaping axes to make iteration easier

axes = axes.reshape(-1)

# Iterating through axes

for idx, ax in enumerate(axes):
    
    # Setting player
    
    player = player_list[idx]
    
    # Setting uniform x and y axes limits

    ax.set_xlim(-.05, 1.05)
    ax.set_ylim(-.05, 1.05)
    
    # Filtering for player and not player

    conds = plot_df.player == player

    player_df = plot_df[conds]

    conds = plot_df.player != player
    
    others_df = plot_df[conds]
    
    # Colors for other player scatter

    color = '#D3D3D3'

    edge_color = 'white'
    
    # Other players scatter

    ax.scatter(x = others_df.xgf_perc, y = others_df.ozf_perc, c = color, edgecolors = edge_color, alpha = .8,
                s = others_df.ixG * 200, zorder = -1)
    
    # Filtering to plot the games where player did not score

    player_plot = player_df[player_df.G == 0]
    
    # Setting non-goal scoring colors

    color = colors['SHOT']

    edge_color = 'white'
    
    # Non goal scoring games scatter

    ax.scatter(x = player_plot.xgf_perc, y = player_plot.ozf_perc, c = color, edgecolors = edge_color, alpha = .8,
                s = player_plot.ixG * 200, zorder = 2)
    
    # Filtering for games where player scored

    player_plot = player_df[player_df.G > 0]
    
    # Colors for games where player scored

    color = colors['GOAL']

    edge_color = colors['SHOT']
    
    # Goal scoring games scatter

    ax.scatter(x = player_plot.xgf_perc, y = player_plot.ozf_perc, c = color, edgecolors = edge_color, alpha = .8,
                s = player_plot.ixG * 200, zorder = 3)
    
    # Setting horizontal and vertical lines

    ax.axhline(y = .5, zorder = 1, c = '#A9A9A9', alpha = 0.8, lw = .9)

    ax.axvline(x = .5, zorder = 1, c = '#A9A9A9', alpha = 0.8, lw = .9)
    
    # Setting axes ticks and labels, but only on certain plots
    
    y_list = [0, 3, 6, 9]
    
    if idx not in y_list:
        
        ax.set_yticklabels('')
        
        ax.set_yticks([])
        
    else:
        
        ax.set_ylabel('OZ FO%', fontsize = 8, labelpad = .5)
        
        ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
    
    x_list = [9, 10, 11]
    
    if idx not in x_list:
        
        ax.set_xticklabels('')
        
        ax.set_xticks([])
        
    else:
        
        ax.set_xlabel('xGF%', fontsize = 8)
        
        ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
        
    # Setting ax title
        
    ax_title = f"{player.replace('.', ' ')}"
        
    ax.set_title(ax_title, fontsize = 8, weight = 'bold')
    
    # Changing size of tick paramters
    
    ax.tick_params(axis='both', which='major', labelsize=7)
    
    # Getting summary statistics
    
    xg = player_df.ixG.sum()
    
    g = player_df.G.sum()
    
    toi = player_df.TOI.sum()
    
    # Text for summary stats box
    
    textstr = f'{round(g)} G |  {round(xg, 2)} xG | {round(toi, 2)} TOI'
    
    # Style for summary stats box
    
    props = dict(boxstyle='round', facecolor='white', alpha=0.9, ec = '#A9A9A9')
    
    # Setting the summary stats box
    
    ax.text(0.5, 0.96, textstr, transform=ax.transAxes, fontsize=5,
        ha = 'center', va = 'center', bbox=props)
    
    # Despining the ax
     
    sns.despine()
    
# Figure title

st_str = 'Duchene & Forsberg driving offense, while Herd deployed defensively'.upper()

fig.suptitle(st_str, ha = 'center', va = 'center', x = 0.5, y = .95, fontsize = 11, weight = 'bold')

# Figure subtitle

stsb_str = '2021-22 5v5 zone usage & offense creation | Data @EvolvingHockey | Viz @chickenandstats'

fig.text(s = stsb_str, ha = 'center', va = 'center', x = .5, y = .925, fontsize = 9)

# Saving figure

file_path = f'./charts/ind_usage/{team}_scatter_5v5.png'

#fig.savefig(file_path, bbox_inches = 'tight', dpi = 500, facecolor = 'white')

## 5. Team xGF and xGA scatter plots

### Function to prep team game and strength level dataframe

In [ ]:
def prep_team_s(data):
    
    '''
    Function to create a team game and strength level data frame
    '''
    
    df = data.copy()
    
    group_list = ['season', 'session', 'game_id', 'game_date', 'event_team', 'opp_team', 'strength_state']

    agg_list = ['CORSI', 'FENWICK', 'GOAL', 'MISS', 'SHOT', 'pred_goal']

    agg_dict = {x: 'sum' for x in agg_list}

    cols = {'event_team': 'team', 'CORSI': 'CF', 'FENWICK': 'FF', 'GOAL': 'GF',
            'MISS': 'MF', 'SHOT': 'SF', 'pred_goal': 'xGF'}

    team_s = df.groupby(group_list, as_index = False).agg(agg_dict).rename(columns = cols)
    
    cols = {'opp_team': 'team', 'CORSI': 'CA', 'FENWICK': 'FA', 'GOAL': 'GA',
        'MISS': 'MA', 'SHOT': 'SA', 'pred_goal': 'xGA'}

    opp = eh_pbp.groupby(group_list, as_index = False).agg(agg_dict).rename(columns = cols)

    merge_list = ['season', 'session', 'game_id', 'game_date', 'team', 'strength_state']

    team_s = team_s.merge(opp, on = merge_list, how = 'outer').fillna(0)

    replace_dict = {'L.A': 'LAK', 'S.J': 'SJS', 'N.J': 'NJD', 'T.B': 'TBL'}

    cols = ['team', 'opp_team']

    for col in cols:

        team_s[col] = team_s[col].map(replace_dict).fillna(team_s[col])
        
    return team_s


### Scraping schedule for winners

In [ ]:

# Setting years

years = list(range(2007, 2022))

# Creating list to collect the schedules

sched_list = []

# Iterating through the years

for year in years:
    
    # Scraping yearly schedule
    
    sched_y = scrape_schedule(year, disable_print = True)
    
    # Appending to schedule list for later concatenation
    
    sched_list.append(sched_y)
    
# Concatenating schedules
    
sched = pd.concat(sched_list, ignore_index = True)

# Creating winner column

winner = np.where(sched.home_team_score > sched.away_team_score, sched.home_team_code, sched.away_team_code)

# Creating winner dictionary 

winner = dict(zip(sched.game_id, winner))


### Munging data and adding win/loss columns

In [ ]:
team_s = prep_team_s(eh_pbp)

team_s['winner'] = team_s.game_id.map(winner)

team_s['win'] = np.where(team_s.team == team_s.winner, 1, 0)

team_s['loss'] = np.where(team_s.team == team_s.winner, 0, 1)


### Divisions and division names

In [ ]:
divisions = {'MET': ['CAR', 'CBJ', 'NJD', 'NYI', 'NYR', 'PHI', 'PIT', 'WSH'],
             'ATL': ['BOS', 'BUF', 'DET', 'FLA', 'MTL', 'OTT', 'TBL', 'TOR'],
             'CEN': ['ARI', 'CHI', 'COL', 'DAL', 'MIN', 'NSH', 'STL', 'WPG'],
             'PAC': ['ANA', 'CGY', 'EDM', 'LAK', 'SJS', 'SEA', 'VAN', 'VGK']}

TEAMS_DICT = dict(zip(sched.home_team_code.unique(), sched.home_team_name.unique()))

DIVISIONS_DICT = {'MET': 'METROPOLITAN', 'ATL': 'ATLANTIC', 'CEN': 'CENTRAL', 'PAC': 'PACIFIC'}

### Plotting data

In [ ]:

strengths = ['5v5']

season = 20212022

session = 'R'

conds = [team_s.strength_state.isin(strengths),
         team_s.season == season,
         team_s.session == session]

conds = np.logical_and.reduce(conds)
    
plot_df = team_s[conds]

for division, teams in divisions.items():
    
    t_order = plot_df[plot_df.team.isin(teams)].groupby('team', as_index = False).agg({'win': 'sum'}).sort_values(by = 'win', ascending = False)
    
    teams_list = list(t_order.team)
    
    fig, axes = plt.subplots(nrows = 4, ncols = 2, figsize = (8, 16), dpi = 500)
    
    sns.despine()

    axes = axes.reshape(-1)
    
    for idx, ax in enumerate(axes):
        
        team = teams_list[idx]
        
        ax.set_xlim(0, 6)
        ax.set_ylim(0, 6)
        
        size = 40
        
        colors = NHL_COLORS[team]
        
        other_conds = plot_df.team != team
        
        color = colors['MISS']
        
        edge_color = 'white'
        
        ax.scatter(y = plot_df[other_conds].xGF, x = plot_df[other_conds].xGA, color = color, ec = edge_color,
                   s = size, lw = .8, alpha = 0.8, zorder = 1)
        
        conds = np.logical_and(plot_df.team == team, plot_df.win == 0)
        
        color = colors['SHOT']
        
        edge_color = 'white'
        
        ax.scatter(y = plot_df[conds].xGF, x = plot_df[conds].xGA, color = color, ec = edge_color,
                   s = size, lw = .8, alpha = 0.8, zorder = 3)
        
        conds = np.logical_and(plot_df.team == team, plot_df.win == 1)
        
        color = colors['GOAL']
        
        edge_color = colors['SHOT']
        
        ax.scatter(y = plot_df[conds].xGF, x = plot_df[conds].xGA, color = color, ec = edge_color,
                   s = size, lw = .8, alpha = 0.8, zorder = 4)

        ax_title = f"{TEAMS_DICT[team]}"

        ax.set_title(ax_title, fontsize = 8, weight = 'bold')
        
        ax.set_ylabel('5v5 xG FOR', fontsize = 7)
        
        ax.set_xlabel('5v5 xG AGAINST', fontsize = 7, labelpad = .4)
            
        ax.tick_params(axis='both', which='major', labelsize=6)
        
        ax.plot([0, 1], [0, 1], transform=ax.transAxes, c = '#A9A9A9', alpha = 0.8, lw = .9, zorder = 2)
        
        stats = plot_df[plot_df.team == team]
        
        gf = stats.GF.sum()
        
        xgf = stats.xGF.sum()
        
        ga = stats.GA.sum()
        
        xga = stats.xGA.sum()
        
        w = stats.win.sum()
        
        l = stats.loss.sum()

        textstr = f'{round(w)} W - {round(l)} L | {round(xgf, 2)} xGF - {round(xga, 2)} xGA'

        props = dict(boxstyle='round', facecolor='white', alpha=0.9, lw = .8, ec = 'white')

        ax.text(0.5, .99, textstr, transform=ax.transAxes, fontsize=7,
            ha = 'center', va = 'center', bbox=props)
        
        win_ball = Line2D([0], [0], color=colors['SHOT'], markerfacecolor = colors['GOAL'], lw=0,
                          label='WIN', markersize = 7, marker = 'o')
        
        loss_ball = Line2D([0], [0], color=colors['SHOT'], markerfacecolor = colors['SHOT'], lw=0,
                           label='LOSS', markersize = 7, marker = 'o')
        
        oth_ball = Line2D([0], [0], color=colors['MISS'], markerfacecolor = colors['MISS'], lw=0,
                          label='OTHER TEAMS', markersize = 7, marker = 'o')
        
        legend_elements = [win_ball, loss_ball, oth_ball]

        ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor = (.5, .96), fontsize = 5, ncol = 3, borderpad = .55)
        
    st_str = f"{DIVISIONS_DICT[division]} DIVISION"
        
    fig.suptitle(st_str, fontsize = 12, weight = 'bold', x = .5, y = .925, ha = 'center')
    
    sub_str = "2021-2022 5v5 offense & defense | Data @EvolvingHockey| Viz @chickenandstats"
    
    fig.text(s = sub_str, x = .5, y = .905, ha = 'center', fontsize = 9)
    
    file_path = f"./charts/team_xg/{division}_5v5.png"
    
    #fig.savefig(file_path, dpi = 500, bbox_inches = 'tight', facecolor = 'white')
        
        

## 6. Rolling xG

### Function for rolling data

In [ ]:
def get_xG_rolling_data(data, season, session, team, strengths, window=10):
    '''
    This function returns xG rolling average figures for a specific team.
    '''
    df = data.copy()
    
    conds = [df.season == season, df.session == session, df.team == team]
    
    game_num = df[np.logical_and.reduce(conds)].game_id.unique()
    
    num_map = {x: idx + 1 for idx, x in enumerate(game_num)}
    
    conds = [df.season == season, df.session == session, df.team == team, df.strength_state.isin(strengths)]
    df = df[np.logical_and.reduce(conds)].copy()
    
    df['game_num'] = df.game_id.map(num_map)
    
    for_list = ['CF', 'FF', 'SF', 'GF', 'xGF'] 
    against_list = ['CA', 'FA', 'SA', 'GA', 'xGA']
    
    stats_dict = dict(zip(for_list, against_list))
    
    for f, a in stats_dict.items():
        
        df[f'rolling_{f}'] = df[f].rolling(window=window, min_periods=0).mean()
        
        df[f'rolling_{a}'] = df[a].rolling(window=window, min_periods=0).mean()
        
        df[f'rolling_{f}_diff'] = df[f'rolling_{f}'] - df[f'rolling_{a}']
    
    return df

### Plotting data

In [ ]:

# Bringing back the divisions

divisions = {'MET': ['CAR', 'CBJ', 'NJD', 'NYI', 'NYR', 'PHI', 'PIT', 'WSH'],
             'ATL': ['BOS', 'BUF', 'DET', 'FLA', 'MTL', 'OTT', 'TBL', 'TOR'],
             'CEN': ['ARI', 'CHI', 'COL', 'DAL', 'MIN', 'NSH', 'STL', 'WPG'],
             'PAC': ['ANA', 'CGY', 'EDM', 'LAK', 'SJS', 'SEA', 'VAN', 'VGK']}

# Setting filder conditions

year = 20212022

session = 'R'

strengths = ['5v5']

for division, teams in divisions.items():
    
    # Getting proper order of teams
    
    order_df = team_s.drop_duplicates(subset = ['team', 'season', 'session', 'win', 'game_id'])
    
    order_conds = [order_df.team.isin(teams), order_df.season == year, order_df.session == session]
    
    t_order = order_df[np.logical_and.reduce(order_conds)].groupby(['season', 'session', 'team'], as_index = False)\
                        .agg({'win': 'sum'}).sort_values(by = 'win', ascending = False)
    
    teams_list = list(t_order.team)
    
    # Generating figure and subplots
    
    fig, axes = plt.subplots(nrows = 4, ncols = 2, figsize = (8, 16), dpi = 500)
    
    # Removing top and right spines
    
    sns.despine()
    
    # Reshaping axes to iterate easier

    axes = axes.reshape(-1)
    
    # Iterating through axes
    
    for idx, ax in enumerate(axes):
        
        # Setting team
        
        team = teams_list[idx]
        
        # Setting uniform y limit
        
        ax.set_ylim(.75, 3.25)
        
        # Getting df for plotting

        df = get_xG_rolling_data(team_s, year, session, team, strengths, window = 10)
        
        # Getting the Y data to plot

        Y_for= df.rolling_xGF.copy().reset_index(drop = True)
        Y_ag = df.rolling_xGA.copy().reset_index(drop = True)
        
        # Getting the X data to plot

        X = pd.Series(range(1, max(df.game_num) + 1))
        
        # Setting colors
        
        colors = NHL_COLORS[team]

        for_c = colors['GOAL']
        
        ag_c = colors['SHOT']
        
        # Setting path effects for xGF line
        
        if for_c == '#FFFFFF':
            
            pe_ec = ag_c
            
        else:
            
            pe_ec = 'white'
            
        pe_for = [mpe.Stroke(linewidth=3.25, foreground=for_c), mpe.Stroke(foreground=pe_ec,alpha=1, linewidth = 4), mpe.Normal()]
        
        # Plotting xGF

        sns.lineplot(x = X, y = Y_for, color = for_c, ax = ax, zorder = 3, path_effects = pe_for)
        
        # Setting path effects for xGA line
        
        if ag_c == '#FFFFFF':
            
            pe_ec = for_c
            
        else:
            
            pe_ec = 'white'
            
        pe_ag = [mpe.Stroke(linewidth=3.25, foreground=ag_c), mpe.Stroke(foreground=pe_ec,alpha=1, linewidth = 4), mpe.Normal()]
        
        # Plotting xGA line
        
        sns.lineplot(x = X, y = Y_ag, color = ag_c, ax = ax, zorder = 3, path_effects = pe_ag)
        
        # Changing colors if for color is white
        
        if for_c == '#FFFFFF':
            
            # Filling between lines
            
            ax.fill_between(
                X, 
                Y_ag,
                Y_for, 
                where = Y_for > Y_ag, 
                interpolate = True,
                alpha = 0.9,
                zorder = 2,
                facecolor = for_c,
                edgecolor = ag_c,
                hatch = '/////',
                lw = 1)
            
            # Setting path effect for legend
            
            pe_for = [mpe.Stroke(linewidth=3.25, foreground=for_c), mpe.Stroke(foreground=ag_c,alpha=1, linewidth = 4), mpe.Normal()]
            
            # Setting the legend figures
            
            xgf_fill = patches.Patch(facecolor=for_c, edgecolor = ag_c, hatch='/////', label = '+xG DIFFERENTIAL')
            
            xgf_l = patches.Patch(facecolor=for_c, label = 'xG FOR', edgecolor = ag_c)
            
        else:
            
            # Fill between the lines
            
            ax.fill_between(
                X, 
                Y_ag,
                Y_for, 
                where = Y_for > Y_ag, 
                interpolate = True,
                alpha = 0.9,
                zorder = 2,
                color = for_c
            )
            
            # Setting the legend figures
            
            xgf_fill = patches.Patch(facecolor=for_c, edgecolor = for_c, label = '+xG DIFFERENTIAL')
            
            xgf_l = patches.Patch(facecolor=for_c, label = 'xG FOR', edgecolor = for_c)
        
        if ag_c == '#FFFFFF':
            
            # Fill between the lines
            
            ax.fill_between(
                X, 
                Y_ag,
                Y_for, 
                where = Y_ag >= Y_for, 
                interpolate = True,
                alpha = 0.9,
                zorder = 2,
                edgecolor = for_c,
                facecolor = ag_c,
                hatch = '////',
                lw = 1
        )
            
            # Setting the legend figures
            
            xga_fill = patches.Patch(facecolor=ag_c, edgecolor = for_c, hatch='/////', label = '-xG DIFFERENTIAL')
            
            xga_l = patches.Patch(facecolor=ag_c, label = 'xG AGAINST', edgecolor = for_c)
            
        else:
            
            # Fill between the lines
            
            ax.fill_between(
                X, 
                Y_ag,
                Y_for, 
                where = Y_ag >= Y_for, 
                interpolate = True,
                alpha = 0.9,
                zorder = 2,
                color = ag_c,
            )
            
            # Setting the legend figures
            
            xga_fill = patches.Patch(facecolor=ag_c, edgecolor = ag_c, label = '-xG DIFFERENTIAL')
            
            xga_l = patches.Patch(facecolor=ag_c, label = 'xG AGAINST', edgecolor = ag_c)

        sns.despine()
        
        # Ax title
        
        ax_title = f"{TEAMS_DICT[team]}"

        ax.set_title(ax_title, fontsize = 8, weight = 'bold')
        
        # Axis labels
        
        ax.set_ylabel('10-GAME ROLLING 5v5 xG', fontsize = 7)
        
        ax.set_xlabel('GAME NUMBER', fontsize = 7)
        
        # Removing x-axis ticks
        
        ax.set_xticks([])
        
        # Changig axis tick sizes
            
        ax.tick_params(axis='both', which='major', labelsize=6)
        
        # Setting y-axis major locator
        
        ax.yaxis.set_major_locator(mtick.MultipleLocator(1))
        
        # Summary stats
        
        stats = df
        
        gf = stats.GF.sum()
        
        xgf = stats.xGF.sum()
        
        ga = stats.GA.sum()
        
        xga = stats.xGA.sum()
        
        w = stats.win.sum()
        
        l = stats.loss.sum()
        
        # Subtitle text

        textstr = f'{round(w)} W - {round(l)} L | {round(xgf, 2)} xGF - {round(xga, 2)} xGA'

        props = dict(boxstyle='round', facecolor='white', alpha=0.9, lw = .8, ec = 'white')

        ax.text(0.5, .99, textstr, transform=ax.transAxes, fontsize=7,
            ha = 'center', va = 'center', bbox=props)
        
        # Legend elements
        
        legend_elements = [xgf_l, xga_l, xgf_fill, xga_fill]

        ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor = (.5, .96), fontsize = 5, ncol = 2, borderpad = .55)
        
    # Figure title and subtitle
        
    st_str = f"{DIVISIONS_DICT[division]} DIVISION"
        
    fig.suptitle(st_str, fontsize = 12, weight = 'bold', x = .5, y = .925, ha = 'center')
    
    sub_str = "2021-2022 5v5 offense & defense | Data @EvolvingHockey | Viz @chickenandstats"
    
    fig.text(s = sub_str, x = .5, y = .905, ha = 'center', fontsize = 9)
    
    # Saving figure
    
    file_path = f"./charts/rolling_xg/{division}_5v5_rolling_xg.png"
    
    #fig.savefig(file_path, dpi = 500, bbox_inches = 'tight', facecolor = 'white')



## 7. Moving line charts

### Function to prep strength level dataframe

In [ ]:
def munge_ind_s(data):
    
    '''
    Function to clean and prep dataframes for use with moving line chart
    '''
    
    df = data.copy()
    
    pp_list = ['5v4', '5v3', '4v3']

    df['ss_2'] = np.where(df.strength_state.isin(pp_list), 'pp', df.strength_state)
    
    group_list = ['player', 'session', 'season', 'team', 'ss_2']

    cols = ['G', 'ixG', 'GaX']

    for col in cols: 

        df[f'{col.lower()}_cs'] = df.groupby(group_list)[col].transform('cumsum')

    keep_list = ['season', 'session', 'player', 'team', 'game_id']

    group_list = ['season', 'session', 'player', 'team']

    df['game_num'] = df[keep_list].drop_duplicates().groupby(group_list).transform('cumcount') + 1

    df.game_num = df.game_num.fillna(method = 'ffill').astype(int)

    return df

### Function to prep game level dataframe

In [ ]:
def munge_ind_g(data):
    
    '''
    Function to clean and munge individual game dataframe
    
    '''
    
    df = data.copy()
    
    group_list = ['player', 'session', 'season', 'team']

    cols = ['G', 'ixG', 'GaX']

    for col in cols: 

        df[f'{col.lower()}_cs'] = df.groupby(group_list)[col].transform('cumsum')
        
    group_list = ['season', 'session', 'player', 'team']

    df['game_num'] = df.groupby(group_list).transform('cumcount') + 1
    
    return df
    

### Function to add colors

In [ ]:
def add_color_cols(data, team, names, colors):
    
    df = data.copy()
    
    color_dict = dict(zip(names, colors))
    
    color_dict = {name: NHL_COLORS[team][color] for name, color in color_dict.items()}
    
    df['color'] = df.player.map(color_dict).fillna(color_dict['OTHER PLAYERS'])

    df['name_2'] = np.where(df.player.isin(color_dict.keys()), df.player, 'OTHER PLAYERS')
    
    return df

### Munging data

In [ ]:
ind_game = munge_ind_g(ind_game)

In [ ]:
ind_strength = munge_ind_s(ind_strength)

### Adding colors

In [ ]:
team = 'NSH'

names = ['FILIP.FORSBERG', 'MATT.DUCHENE', 'OTHER PLAYERS']

colors = ['GOAL', 'SHOT', 'MISS']

year = 20212022

session = 'R'

In [ ]:
ind_game = add_color_cols(ind_game, team, names, colors)

In [ ]:
ind_strength = add_color_cols(ind_strength, team, names, colors)

### Plotting

In [ ]:
conds = np.logical_and.reduce([ind_game.session == session,
                               ind_game.team == team,
                               ind_game.season == season])

game_stuff = ind_game[conds].copy()

conds = np.logical_and.reduce([ind_strength.session == session,
                               ind_strength.team == team,
                               ind_strength.season == season])

strength_stuff = ind_strength[conds].copy()

game_dates = list(game_stuff.game_date_dt.sort_values().unique())

fig, ax = plt.subplots(3, figsize = (8, 12), dpi = 400,  facecolor = 'white')

fig.tight_layout(pad = 3)

camera = Camera(fig)

for idx, game_date in enumerate(game_dates):
    
    num = idx + 1
    
    ## ALl strengths plotting
    
    chart = game_stuff[game_stuff.game_date_dt <= game_date]

    grays = chart[chart.name_2 == 'OTHER PLAYERS']

    scorers = chart[chart.name_2 != 'OTHER PLAYERS']

    grays_pal = dict(zip(grays.player, grays.color))

    scorers_pal = dict(zip(scorers.player, scorers.color))

    sns.lineplot(data = grays, x = 'game_num', y = 'gax_cs', hue = 'player',
                 palette = grays_pal, legend = None, estimator = None, ax = ax[0], linewidth = 3)

    sns.lineplot(data = scorers, x = 'game_num', y = 'gax_cs', hue = 'player',
                 palette = scorers_pal, legend = None, estimator = None, ax = ax[0], linewidth = 6)
    
    ax[0].set_ylabel('Cumulative goals above expected')
    
    ax[0].set_xlabel('Games played')
    
    for player in scorers.player.unique():
        
        if player == 'FILIP.FORSBERG':
            
            player_num = 'NSH9'
            
        elif player == 'MATT.DUCHENE':
            
            player_num = 'NSH95'
            
        y = scorers[scorers.player == player].gax_cs.iloc[-1]
        
        x = scorers[scorers.player == player].game_num.iloc[-1]
        
        color = scorers[scorers.player == player].color.iloc[0]
        
        text = f"{player_num}" 
        
        ax[0].annotate(text, (x, y), ha = 'left', size = 9, va = 'center',
                    bbox=dict(boxstyle = 'round', fc = 'white', ec = color, alpha = 0.9, pad = .5, lw = 2.5))
        
    ax[0].set_xlabel('')
        
    ax[0].set_title('All situations', size = 10, weight = 'heavy')
        
    ## 5v5 plotting

    chart = strength_stuff[np.logical_and(strength_stuff.game_date_dt <= game_date,
                                          strength_stuff.strength_state == '5v5')]

    grays = chart[chart.name_2 == 'OTHER PLAYERS']

    scorers = chart[chart.name_2 != 'OTHER PLAYERS']

    grays_pal = dict(zip(grays.player, grays.color))

    scorers_pal = dict(zip(scorers.player, scorers.color))

    sns.lineplot(data = grays, x = 'game_num', y = 'gax_cs', hue = 'player',
                 palette = grays_pal, legend = None, estimator = None, ax = ax[1], linewidth = 3)

    sns.lineplot(data = scorers, x = 'game_num', y = 'gax_cs', hue = 'player',
                 palette = scorers_pal, legend = None, estimator = None, ax = ax[1], linewidth = 6)

    ax[1].set_ylabel('Cumulative goals above expected')

    ax[1].set_xlabel('Games played')

    for player in scorers.player.unique():

        if player == 'FILIP.FORSBERG':

            player_num = 'NSH9'

        elif player == 'MATT.DUCHENE':

            player_num = 'NSH95'

        y = scorers[scorers.player == player].gax_cs.iloc[-1]

        x = scorers[scorers.player == player].game_num.iloc[-1]

        color = scorers[scorers.player == player].color.iloc[0]

        text = f"{player_num}" 

        ax[1].annotate(text, (x, y), ha = 'left', size = 9, va = 'center',
                    bbox=dict(boxstyle = 'round', fc = 'white', ec = color, alpha = 0.9, pad = .5, lw = 2.5))
        
    ax[1].set_xlabel('')
        
    ax[1].set_title('5v5', size = 10, weight = 'heavy')
        
    ## Power play plotting

    chart = strength_stuff[np.logical_and(strength_stuff.game_date_dt <= game_date,
                                          strength_stuff.ss_2 == 'pp')]

    grays = chart[chart.name_2 == 'OTHER PLAYERS']

    scorers = chart[chart.name_2 != 'OTHER PLAYERS']

    grays_pal = dict(zip(grays.player, grays.color))

    scorers_pal = dict(zip(scorers.player, scorers.color))

    sns.lineplot(data = grays, x = 'game_num', y = 'gax_cs', hue = 'player',
                 palette = grays_pal, legend = None, estimator = None, ax = ax[2], linewidth = 3)

    sns.lineplot(data = scorers, x = 'game_num', y = 'gax_cs', hue = 'player',
                 palette = scorers_pal, legend = None, estimator = None, ax = ax[2], linewidth = 6)

    ax[2].set_ylabel('Cumulative goals above expected')

    ax[2].set_xlabel('Games played')

    for player in scorers.player.unique():

        if player == 'FILIP.FORSBERG':

            player_num = 'NSH9'

        elif player == 'MATT.DUCHENE':

            player_num = 'NSH95'

        y = scorers[scorers.player == player].gax_cs.iloc[-1]

        x = scorers[scorers.player == player].game_num.iloc[-1]

        color = scorers[scorers.player == player].color.iloc[0]

        text = f"{player_num}" 

        ax[2].annotate(text, (x, y), ha = 'left', size = 9, va = 'center',
                    bbox=dict(boxstyle = 'round', fc = 'white', ec = color, alpha = 0.9, pad = .5, lw = 2.5))
        
    ax[2].set_title('Power play', size = 10, weight = 'heavy')
    
    title_str = "Forsberg was Nashville's best scorer, Duchene reliable on the power play"
    
    fig.text(s = title_str, fontsize = 12, ha = 'center', va = 'center', x = 0.5, 
                     y = 1.03, weight = 'heavy')
    
    sub_str = "2021-2022 Nashville Predators | Data @EvolvingHockey | Viz @chickenandstats"

    fig.text(s = sub_str , size = 11, ha = 'center', va = 'center', x = 0.5, y = 1.01)
    
    camera.snap()
    
mv_stuff = camera.animate(blit = False, repeat_delay = 5000, interval = 150)  
mv_stuff.save(f'./charts/moving_lines/{team}_moving_line_hd.gif', savefig_kwargs = {'facecolor': 'white'})